In [ ]:
%load_ext autoreload
%autoreload 2

# default_exp serving.ubiops

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# prcvd.serving.ubiops

> For code specific to ubiops deployments

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os
import ubiops
from mod.interfaces.types import check_collection
from mod.serving.client import (RequestData, Client)

In [ ]:
#export

# keys of this are from ubiops service definitions
# See: Create new deployment > Input > JSON Editor
ubiops_type_mapping = {
        float: {
            "label": "Double precision",
            "value": "double"
        },
        int: {
            "label": "Integer",
            "value": "int"
        },
        str: {
            "label": "String",
            "value": "string"
        },
        bool: {
            "label": "Boolean",
            "value": "bool"
        },
        "array_int": {
            "label": "Array of integers",
            "value": "array_int"
        },
        "array_float": {
            "label": "Array of doubles",
            "value": "array_double"
        },
        "array_str": {
            "label": "Array of strings",
            "value": "array_string"
        },
        "none": None
    }


def make_ubiops_spec(name, val, type_mapping):
    """
    Takes a key and example value (val) from an 
    output dict and infers the ubiops type definition.
    
    Example:
    ```python
    input_spec = [
        make_ubiops_spec(
            name=k,
            val=v, 
            type_mapping=ubiops_type_mapping)
        for k, v in out['row'].items()
        # if v
    ]
    
    ## out
    output_spec = [
        {'name': 'rgb_of_Background/undefined',
         'data_type': {'label': 'Array of integers', 'value': 'array_int'}},
        {'name': 'rgb_of_Lips',
         'data_type': {'label': 'Array of integers', 'value': 'array_int'}},
         ...]

    ```
    """
    
    if not val:
        t = 'none'
        
    elif seq_not_str(obj=val):
        t = check_collection(val=val)
    
    else:
        t = type(val)
    
    return {
        'name': name,
        'data_type': type_mapping[t]
    }


class UbiOpsInputRequestData(RequestData):
    def __init__(self, spec:dict, 
                 api:ubiops.api.core_api.CoreApi, 
                 project_name:str):
        """Factory class for ubi ops requests."""
        super().__init__(spec=spec)
        self.api = api
        self.project_name = project_name
    
    def _tx_blob(self, v):
        """Returns blob value to be sent in request."""
        b = self.api.blobs_create(
            project_name=self.project_name, file=v
        )
        return b.id


class UbiOpsOutputRequestData(RequestData):
    def __init__(self, spec:dict, 
                 api:ubiops.api.core_api.CoreApi, 
                 project_name:str):
        """Factory class for ubi ops requests."""
        super().__init__(spec=spec)
        self.api = api
        self.project_name = project_name
    
    
    def _tx_blob(self, v):
        """Returns blob value to be sent in request."""
        # recieves blob id
        data = self.api.blobs_get(
            project_name=self.project_name,
            blob_id=v
        )
        self.api.blobs_delete(
            project_name=self.project_name,
            blob_id=v
        )
        return data
    
    
class UbiOpsClient(Client):
    def __init__(self, project_name:str, deployment_name:str, 
                 deployment_version:str, input_spec:dict, 
                 output_spec:dict, api_key:str, api_host:str):
        super().__init__(project_name=project_name, 
                         deployment_name=deployment_name, 
                         deployment_version=deployment_version,
                         input_spec=input_spec, 
                         output_spec=output_spec, api_key=api_key,
                         api_host=api_host)
        self.url_fmt = '{host}/projects/{proj}/'
        self.req_fmt = 'deployments/{dname}/versions/{dver}/request'
        self.req_url = os.path.join(
            self.url_fmt.format(
                host=self.api_host, 
                proj=self.project_name),
            self.req_fmt.format(
                dname=self.deployment_name, 
                dver=self.deployment_version
            )
        )
        self.client = ubiops.ApiClient(
            ubiops.Configuration(
                api_key={'Authorization': self.api_key}, 
                host=self.api_host)
        )
        self.api = ubiops.CoreApi(self.client)
        self.input_factory = UbiOpsInputRequestData(
            spec=self.input_spec, 
            api=self.api, 
            project_name=self.project_name
        )
        self.output_factory = UbiOpsOutputRequestData(
            spec=self.output_spec, 
            api=self.api, 
            project_name=self.project_name
        )
        

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_client.ipynb.
Converted 13_interfaces_types.ipynb.
Converted 14_serving_ubiops.ipynb.
Converted 15_exceptions.ipynb.
Converted 16_serving_client.ipynb.
Converted index.ipynb.
